# Calibration

## what is it?
High frequency measurements require the measurement system
to be calibrated

To calibrate is to:
    
 *Remove the unwanted electrical effects of 
 intermediary circuitry between the internals
 of the VNA and the test ports.*
 
<center><img src="figures/vna.jpg" width="400">
    
But is more general than this. can be used to de-embed anywhere you can have known responses. 
    

## How many types are there?
Most common calibration algorithms can be categorized as
follows
*  One-port (OSL, SDDL, PHN,...)
* Two-port   (TRL, LRM, TRM, ...)
    * 16-term
    * 12-term
    * 8-term  
* N-port

Other things exist;

* Multi-line TRL (Multi-cal)
* iterative solutions (Statisti-Cal)
* multi-frequency

See http://emlab.uiuc.edu/ece451/appnotes/Rytting_NAModels.pdf for more info


## Calibration in skrf
 
**scikit-rf** can be used to calibrate data taken from a VNA assuming you, 
* have measured an acceptable set of standards,
* have a corresponding set ideal responses.

This may be referred to as *offline* calibration, because it is not occurring onboard the VNA itself. 

This technique provides maximum flexibility for non-conventional calibrations, and preserves all raw data.

There are many calibration algorithms implemented in scikit-rf. See the [calibration API's page](https://scikit-rf.readthedocs.io/en/latest/api/calibration/index.html).


## Creating a Calibration

Calibrations are performed through a `Calibration` class. In General, `Calibration` objects require two arguments:

*  a list of measured `Network`'s
*  a list of ideal `Network`'s

The `Network` elements in each list: 
* must all be similar (same number of ports, frequency info, etc)
* must be aligned to each other: the first element of ideals list must correspond to the first element of measured list.

*Self-calibration* algorithms, such as TRL, do not require predefined ideal responses.

## One-Port Example

To construct a one-port calibration, you need to
* have measured at least three standards
* have their known *ideal* responses in the form of `Network`s. 

These `Network` can be created from touchstone files or from ideal analytic's reponse.

The measured and ideal touchstone files for a conventional short-open-load (SOL) calkit are in folders `measured/` and `ideal/`, respectively. These are used to create a `OnePort` Calibration and correct a measured DUT 

In [ ]:
import skrf as rf
from skrf.calibration import OnePort


d= 'data/calibration/oneport/'
# a list of Network types, holding 'ideal' responses
my_ideals = [rf.Network(d+'ideals/short.s1p'),
             rf.Network(d+'ideals/open.s1p'),
             rf.Network(d+'ideals/load.s1p')]

# a list of Network types, holding 'measured' responses
my_measured = [rf.Network(d+'measured/short.s1p'),
               rf.Network(d+'measured/open.s1p'),
               rf.Network(d+'measured/load.s1p')]

In [ ]:
## create a Calibration instance
cal = rf.OnePort( ideals = my_ideals,
                  measured = my_measured)

cal

In [ ]:
# run calibration algorithm
cal.run()

# apply it to a dut
dut = rf.Network(d+'duts/dut.s1p')
dut_caled = cal.apply_cal(dut)
dut_caled.name =  dut.name + ' corrected'

# plot results
from skrf.plotting import stylely;stylely()

dut.plot_s_db()
dut_caled.plot_s_db()

In [ ]:
cal.error_ntwk.plot_s_db()

In [ ]:
cal

## Sloppy One-port
A quick and sloppy way to do a cal 

In [ ]:
cal = OnePort(ideals = rf.read_all(d+'ideals'),
              measured=rf.read_all(d+'measured'))

In [ ]:
cal.ideals, cal.measured

## One-port Tiered 

A one-port network analyzer can be used to measure a two-port device,
provided that the device is reciprocal. 

This is accomplished by performing two calibrations, which is why its called a *tiered* calibration. 

First, the VNA is calibrated at the test-port like normal. This is called the *first tier*. 

Next, the device is connected to the test-port, and a calibration is performed at the far end of the device, the *second tier*. 

<center><img src="figures/boxDiagram.svg" width="800">


We'll use  data that was taken to characterize a waveguide-to-CPW probe. 

For this specific example the diagram above looks like:
<center><img src="figures/probe.svg" width="600">


In [ ]:
ls data/calibration/oneport_tiered_calibration/


In each folder you will find the two sub-folders, called `'ideals/' ` and `'measured/'`. These contain touchstone files of the calibration standards ideal and measured responses,  respectively. 

The first tier is at waveguide interface, and consisted of the following set of standards

* short 
* delay short
* load
* radiating open (literally an open waveguide)

### Tier 1

In [ ]:

d = 'data/calibration/oneport_tiered_calibration/'
tier1_ideals   = rf.read_all_networks(d+'/tier1/ideals/')
tier1_measured = rf.read_all_networks(d+'/tier1/measured/')
 

tier1 = OnePort(measured = tier1_measured,
                ideals = tier1_ideals,
                name = 'tier1',
                sloppy_input=True)
tier1

### Tier 2

In [ ]:
tier2_ideals = rf.read_all_networks(d+'/tier2/ideals/')
tier2_measured = rf.read_all_networks(d+'/tier2/measured/')
 

tier2 = OnePort(measured = tier2_measured,
                ideals = tier2_ideals,
                name = 'tier2',
                sloppy_input=True)
tier2

The error network for *tier1* models the VNA

the error network for *tier2* represents the VNA+DUT. 

To determine the DUT's response, we cascade the inverse S-parameters of the VNA with the VNA+DUT. 

$$ DUT = VNA^{-1}\cdot (VNA \cdot DUT)$$

<center><img src="figures/probe.svg" width="600">

In [ ]:
from matplotlib import pyplot as plt
dut = tier1.error_ntwk.inv ** tier2.error_ntwk
dut.name = 'probe'
dut.plot_s_db()
plt.title('Probe S-parameters')
plt.ylim(-60,10);

##  Two-port Calibrations
**skrf** supports a few different two-port algorithms: 

* 12-term error model, such as the traditional `SOLT`. Similar to the OnePort example.
* The `EightTerm` calibrations can  be constructed from any number of standards, providing that:
 * you need 3 **two-port** standards:
   * one must be transmissive, 
   * one must provide a known impedance 
   * one must be reflective.
 * Important detail:  of using the 8-term error model formulation is that switch-terms may need to be measured in order to achieve a high quality calibration
 
Note, that the word 8-term is used in the literature to describe a specific error model used by a variety of calibration algorithms, like TRL, LRM, etc. The EightTerm class, is an implementation of the algorithm cited above, which does not use any self-calibration.

## TRL

In [ ]:
from skrf.calibration import TRL

d = 'data/calibration/trl_data/'
T = rf.Network(d+'thru.s2p')
R = rf.Network(d+'reflect.s2p')
L = rf.Network(d+'line.s2p')

switch_terms = (rf.Network(d+'forward switch term.s1p'),
                rf.Network(d+'reverse switch term.s1p'))

measured = [T,R,L]

In [ ]:
trl = TRL(measured = measured, 
          switch_terms = switch_terms)
trl

In [ ]:
dut_raw = rf.Network(d+'mismatched line.s2p')
dut_corrected = trl.apply_cal(dut_raw)
dut_corrected.plot_s_db()

## Tip: Using one-port ideals in two-port Calibration

Commonly, you have data for ideal data for reflective standards in the form of one-port touchstone files (ie `.s1p`).

To use this with skrf's two-port calibration method you need to create a two-port network that is a composite of the two networks. The  function `skrf.network.two_port_reflect` does this:

<img src="figures/2_1port_to_1_2port.svg"/>

In [ ]:
short = rf.Network('data/calibration/oneport/ideals/short.s1p')
shorts = rf.two_port_reflect(short, short)
shorts

## Switch-terms

Originally described by Roger Marks, *switch-terms* account for the fact that the 8-term (aka *error-box* ) model is overly simplified:
* The two error networks change slightly depending on which port is being excited due to internal switch within the VNA. 
* Switch terms describe the imperfect load when the source is switch to the other port.

Switch terms can be directly measured with a low insertion loss transmissive standard (like a thru) connected between the cables connected to the instrument ports. 

Sometime, a custom measurement configuration is available on the VNA itself. 

Without switch-term measurements, your calibration quality will vary depending on properties of you VNA.

## SOLT Example

Two-port calibration is accomplished in an identical way to one-port, except all the standards are two-port networks. 

This is even true of reflective standards (S21=S12=0). 

So if you measure reflective standards you must measure two of them simultaneously, and store information in a two-port:

<img src="figures/VNA_2_1port.svg"/>

For example, connect a short to port-1 and a load to port-2, and save a two-port measurement as 'short,load.s2p' or similar
   
    # a list of Network types, holding 'ideal' responses
    my_ideals = [
	    rf.Network('ideal/thru.s2p'),
	    rf.Network('ideal/short, short.s2p'),
	    rf.Network('ideal/open, open.s2p'),
	    rf.Network('ideal/load, load.s2p'),
	    ]
    
    # a list of Network types, holding 'measured' responses
    my_measured = [
	    rf.Network('measured/thru.s2p'),
	    rf.Network('measured/short, short.s2p'),
	    rf.Network('measured/open, open.s2p'),
	    rf.Network('measured/load, load.s2p'),
	    ]

    ## create a SOLT instance
    cal = rf.calibration.SOLT(
	    ideals = my_ideals,
	    measured = my_measured,
	    )

    # run calibration algorithm
    cal.run() 
    
    # apply it to a dut
    dut = rf.Network('my_dut.s2p')
    dut_caled = cal.apply_cal(dut)

# Example : one port tiered calibration?